In [1]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

### Задание №1

Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

Так как у нас зарплата на hh.ru может быть представлена в разных форматах (160 000-250 000 руб., 2 000-3 000 EUR и тп), то ее нужно привести к единому формату, чтобы вдальнейшем можно было производить различные манипуляции с данными. Делаем допущение и зашиваем жестко курс валюты, хотя можно было и его спарсить.

In [2]:
def hh_salary_parse(salary):
    currency = 1
    if 'USD' in salary:
        currency = 63.58
    if 'EUR' in salary:
        currency = 71.66
    if salary == 'Не указанно':
        salary = '0.0'
    salary = salary.replace(' ', '')    
    salary = salary[:salary.find('-')]
    salary = re.findall("\d+", salary)
    salary = float(salary[0]) * currency
    return salary

In [3]:
def hh_parse(vacancy_name, search_depth, analysis_period):
    n = 0
    titles = []
    salarys = []
    salarys_processed = []
    hrefs = []
    
    hh_url = 'https://hh.ru/search/vacancy?search_period='+str(analysis_period)+'&text='+vacancy_name+'&area=1'
    headers = {'accept': '*/*',
           'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15'}
    
    session = requests.session()
    
    while n < search_depth:
        request = session.get(hh_url+'&page='+str(n), headers=headers)
        hh_html = bs(request.content, 'html.parser')
        divs = hh_html.find_all('div', attrs={'data-qa': 'vacancy-serp__vacancy'})
        for div in divs:
            title = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}).text
            salary = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})
            if salary == None:
                salary = 'Не указанно'
            else:
                salary = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text

            href = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href']
            salary_processed = hh_salary_parse(salary) * 1000
            titles.append(title)
            salarys.append(salary)
            salarys_processed.append(salary_processed)
            hrefs.append(href)
        n += 1
    df = pd.DataFrame({'Title': titles,
                       'Salary': salarys,
                       'Salary_Processed': salarys_processed,
                       'Href': hrefs}, columns=['Title', 'Salary', 'Salary_Processed', 'Href'])
    df.replace(u'\xa0', u'', regex=True, inplace=True)
    return df

In [4]:
vacancy_name = input('Введите название вакансии для поиска: ')

Введите название вакансии для поиска: python


In [5]:
search_depth = int(input('Введите глубину поиска (количество страниц): '))

Введите глубину поиска (количество страниц): 2


In [6]:
analysis_period = int(input('Введите период анализа (количество дней): '))

Введите период анализа (количество дней): 10


In [7]:
hh_frame = hh_parse(vacancy_name, search_depth, analysis_period)

In [8]:
hh_frame

,Title,Salary,Salary_Processed,Href
0,Python Developer,160000-250000 руб.,160000.0,https://vidnoe.hh.ru/vacancy/32072763?query=py...
1,Программист Python (в Ригу),2000-3000 EUR,143320.0,https://vidnoe.hh.ru/vacancy/32079548?query=py...
2,Программист Python,75000-110000 руб.,75000.0,https://vidnoe.hh.ru/vacancy/31347220?query=py...
3,Программист Python,60000-100000 руб.,60000.0,https://vidnoe.hh.ru/vacancy/32260756?query=py...
4,Разработчик Python и SQL,от 180000 руб.,180000.0,https://vidnoe.hh.ru/vacancy/31929637?query=py...
5,Python-разработчик,от 100000 руб.,100000.0,https://vidnoe.hh.ru/vacancy/32251802?query=py...
6,Разработчик Python,160000-200000 руб.,160000.0,https://vidnoe.hh.ru/vacancy/31624245?query=py...
7,Python/Django разработчик,60000-100000 руб.,60000.0,https://vidnoe.hh.ru/vacancy/32158934?query=py...
8,Ведущий Python разработчик,150000-250000 руб.,150000.0,https://vidnoe.hh.ru/vacancy/32061372?query=py...
9,Разработчик Python/Django + React.js,145000-255000 руб.,145000.0,https://vidnoe.hh.ru/vacancy/31844761?query=py...


После того как получили данные с hh.ru и вывели их на экран, теперь сохраним их в MongoDB

In [9]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['headhanter']
hhdb = db.headhanter

In [10]:
for index, row in hh_frame.iterrows():
    hh_data={
        'Title': row['Title'],
        'Salary': row['Salary'],
        'Salary_Processed': row['Salary_Processed'],
        'Href': row['Href']
    }
    hhdb.insert(hh_data)

In [11]:
hhdb.headhanter

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'headhanter'), 'headhanter.headhanter')

Проверяем что данные записали в базу, вывод первые 5 записей 

In [12]:
for vacancy in hhdb.find().limit(5):
    pprint(vacancy)

{'Href': 'https://vidnoe.hh.ru/vacancy/32072763?query=python',
 'Salary': '160000-250000 руб.',
 'Salary_Processed': 160000.0,
 'Title': 'Python Developer',
 '_id': ObjectId('5d1fd100f943af417829c7a5')}
{'Href': 'https://vidnoe.hh.ru/vacancy/32079548?query=python',
 'Salary': '2000-3000 EUR',
 'Salary_Processed': 143320.0,
 'Title': 'Программист Python (в Ригу)',
 '_id': ObjectId('5d1fd101f943af417829c7a6')}
{'Href': 'https://vidnoe.hh.ru/vacancy/31347220?query=python',
 'Salary': '75000-110000 руб.',
 'Salary_Processed': 75000.0,
 'Title': 'Программист Python',
 '_id': ObjectId('5d1fd101f943af417829c7a7')}
{'Href': 'https://vidnoe.hh.ru/vacancy/32260756?query=python',
 'Salary': '60000-100000 руб.',
 'Salary_Processed': 60000.0,
 'Title': 'Программист Python',
 '_id': ObjectId('5d1fd101f943af417829c7a8')}
{'Href': 'https://vidnoe.hh.ru/vacancy/31929637?query=python',
 'Salary': 'от 180000 руб.',
 'Salary_Processed': 180000.0,
 'Title': 'Разработчик Python и SQL',
 '_id': ObjectId('5d1

### Задание №2

Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы

In [13]:
salary_threshold = float(input('Введите пороговую зарплату в рублях, выше которой будет осуществлять поиск: '))

Введите пороговую зарплату в рублях, выше которой будет осуществлять поиск: 100000


In [14]:
for vacancy in hhdb.find().sort('Salary_Processed').where('this.Salary_Processed >' + str(salary_threshold)):
    pprint(vacancy)

{'Href': 'https://vidnoe.hh.ru/vacancy/30451399?query=python',
 'Salary': '120000-180000 руб.',
 'Salary_Processed': 120000.0,
 'Title': 'Разработчик Python',
 '_id': ObjectId('5d1fd101f943af417829c7b1')}
{'Href': 'https://vidnoe.hh.ru/vacancy/31974215?query=python',
 'Salary': '120000-200000 руб.',
 'Salary_Processed': 120000.0,
 'Title': 'Разработчик Python/Team-lead',
 '_id': ObjectId('5d1fd101f943af417829c7bc')}
{'Href': 'https://vidnoe.hh.ru/vacancy/30451399?query=python',
 'Salary': '120000-180000 руб.',
 'Salary_Processed': 120000.0,
 'Title': 'Разработчик Python',
 '_id': ObjectId('5d1fd12b99135287c93f0b34')}
{'Href': 'https://vidnoe.hh.ru/vacancy/31974215?query=python',
 'Salary': '120000-200000 руб.',
 'Salary_Processed': 120000.0,
 'Title': 'Разработчик Python/Team-lead',
 '_id': ObjectId('5d1fd12b99135287c93f0b3f')}
{'Href': 'https://vidnoe.hh.ru/vacancy/31965654?query=python',
 'Salary': 'до 130000 руб.',
 'Salary_Processed': 130000.0,
 'Title': 'Python-разработчик',
 '_id